In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [631 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,512 kB]
Get:13 http://archive.ubuntu.com/ubuntu j

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show DataFrame
home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales_df')
spark.sql("SELECT * FROM home_sales_df LIMIT 10").show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
result = spark.sql("""
    SELECT
        date_built AS year,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        home_sales_df
    WHERE
        bedrooms = 4
    GROUP BY
        date_built
    ORDER BY
        date_built
""")

# Show the result
result.show()

+----+---------+
|year|avg_price|
+----+---------+
|2010|296800.75|
|2011| 302141.9|
|2012|298233.42|
|2013|299999.39|
|2014|299073.89|
|2015|307908.86|
|2016|296050.24|
|2017|296576.69|
+----+---------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
result2 =spark.sql("""
  SELECT
    date_built AS year,
    ROUND (AVG(price), 2) AS avg_price
    FROM
      home_sales_df
    WHERE
      bedrooms = 3
      AND
      bathrooms = 3
    GROUP BY
      date_built
    ORDER BY
      date_built
  """)

result2.show()


+----+---------+
|year|avg_price|
+----+---------+
|2010|292859.62|
|2011|291117.47|
|2012|293683.19|
|2013|295962.27|
|2014|290852.27|
|2015| 288770.3|
|2016|290555.07|
|2017|292676.79|
+----+---------+



In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
result3 =spark.sql("""
  SELECT
    date_built AS year,
    ROUND (AVG(price), 2) AS avg_price
    FROM
      home_sales_df
    WHERE
      bedrooms = 3
      AND
      bathrooms = 3
      AND
      floors = 2
      AND
      sqft_lot >= 2000
    GROUP BY
      date_built
    ORDER BY
      date_built
  """)

result3.show()


+----+---------+
|year|avg_price|
+----+---------+
|2010|280447.23|
|2011|281413.45|
|2012|295858.68|
|2013|295142.13|
|2014|294195.13|
|2015|291788.36|
|2016| 287812.6|
|2017|282026.59|
+----+---------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

result4 = spark.sql("""
    SELECT
        ROUND(AVG(price), 2) AS avg_price,
        MAX(view) AS max_view
    FROM
        home_sales_df
    WHERE
        price >= 350000
""")


result4.show()

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

+---------+--------+
|avg_price|max_view|
+---------+--------+
|473796.26|      99|
+---------+--------+

--- 0.00014138221740722656 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales_df")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales_df')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

# Cache the DataFrame
spark.sql("CACHE TABLE home_sales_df")

# Run the Spark SQL query with caching
result_cached = spark.sql("""
    SELECT
        ROUND(AVG(price), 2) AS avg_price,
        MAX(view) AS max_view
    FROM
        home_sales_df
    WHERE
        price >= 350000
""")

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


--- 0.00011038780212402344 seconds ---


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

# Partition the DataFrame by the "date_built" field and write to Parquet
home_sales_df.write.partitionBy("date_built").parquet("/path/to/output/directory")

In [13]:
# 11. Read the parquet formatted data.

# Specify the path to the Parquet data
parquet_path = "/path/to/output/directory"

# Read Parquet data into a DataFrame
df = spark.read.parquet(parquet_path)

# Show the DataFrame
df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [14]:
# 12. Create a temporary table for the parquet data.


df.createOrReplaceTempView("parquet_data")

In [16]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.



start_time = time.time()


# Run the Spark SQL query without caching on Parquet data
result_uncached_parquet = spark.sql("""
    SELECT
        ROUND(AVG(price), 2) AS avg_price,
        MAX(view) AS max_view
    FROM
        parquet_data
    WHERE
        price >= 350000
""")

# Show the result
result_uncached_parquet.show()

# Record the end time for uncached query
end_time_uncached = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

+---------+--------+
|avg_price|max_view|
+---------+--------+
|473796.26|      99|
+---------+--------+

--- 0.6620452404022217 seconds ---


In [18]:
# 14. Uncache the home_sales temporary table.

# Uncache the home_sales temporary table
spark.sql("UNCACHE TABLE home_sales_df")

DataFrame[]

In [20]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales_df")

# Print the result
if is_cached:
    print("The home_sales_df table is still cached.")
else:
    print("The home_sales_df table is not cached.")


The home_sales_df table is not cached.
